# baseline model (large scale)
Be advised: it could take very long

In [ ]:
import logging
import os

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import torch
from torch.distributions import constraints
from torch.autograd import Variable

import pyro
import pyro.distributions as dist
import pyro.optim as optim

from pyro.infer import EmpiricalMarginal, SVI, Trace_ELBO, Predictive
from pyro.contrib.autoguide import AutoMultivariateNormal
from pyro.infer.mcmc.api import MCMC
from pyro.infer.mcmc import NUTS

pyro.set_rng_seed(1)
assert pyro.__version__.startswith('1.4.0')

In [ ]:
anime_data = pd.read_csv('archive/rating.csv').drop_duplicates()
# anime_data.head()
anime_data.shape

In [ ]:
# Prepare the matrix
anime_matrix_df = anime_data.pivot_table(index='user_id', columns='anime_id', values='rating').fillna(0)
#anime_data.groupby(['user_id', 'anime_id'])['rating']first().unstack(fill_value=0.0)
anime_matrix_df.head()

In [ ]:
anime_matrix_df.shape

In [ ]:
# assume ratings of -1 have a normal distribution
# Known paramters
mu = 5
sigma = 1
sigma_u = 1
sigma_v = 1
sigma_1 = 1
sigma_2 = 2

# dimension
k = 50

def fill_neg_one(value):
    if value == -1:
        return round(pyro.sample('r_ab', dist.Normal(mu, sigma_1)).item())
    else:
        return value
    
anime_matrix_df = anime_matrix_df.applymap(fill_neg_one)

In [ ]:
anime_matrix_df.head()

In [ ]:
anime_matrix = anime_matrix_df.values

In [ ]:
user_ratings_mean = np.mean(anime_matrix, axis = 1)
anime_matrix_demeaned = anime_matrix - user_ratings_mean.reshape(-1, 1)

In [ ]:
anime_matrix

In [ ]:
anime_matrix_df.shape

In [ ]:
from scipy.sparse.linalg import svds
U, sigma, Vt = svds(anime_matrix_demeaned, k = 50)

In [ ]:
sigma = np.diag(sigma)

In [ ]:
all_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt) + user_ratings_mean.reshape(-1, 1)
preds_df = pd.DataFrame(all_user_predicted_ratings, columns = anime_matrix_df.columns)

In [ ]:
anime_info = pd.read_csv('archive/anime.csv')
anime_info['anime_id'] = anime_info['anime_id'].apply(pd.to_numeric)
anime_info

In [ ]:
preds_df

In [ ]:
def recommend_movies(predictions_df, userID, movies_df, original_ratings_df, num_recommendations=5):
    
    # Get and sort the user's predictions
    user_row_number = userID - 1 # UserID starts at 1, not 0
    sorted_user_predictions = predictions_df.iloc[user_row_number].sort_values(ascending=False)
    
    # Get the user's data and merge in the movie information.
    user_data = original_ratings_df[original_ratings_df.user_id == (userID)]
    user_full = (user_data.merge(movies_df, how = 'left', left_on = 'anime_id', right_on = 'anime_id')
                 )

    print ('User {0} has already rated {1} movies.'.format(userID, user_full.shape[0]))
    print ('Recommending the highest {0} predicted ratings movies not already rated.'.format(num_recommendations))
    
    # Recommend the highest predicted rating movies that the user hasn't seen yet.
    recommendations = (movies_df[~movies_df['anime_id'].isin(user_full['anime_id'])].
         merge(pd.DataFrame(sorted_user_predictions).reset_index(), how = 'left',
               left_on = 'anime_id',
               right_on = 'anime_id').
         rename(columns = {user_row_number: 'Predictions'}).
         sort_values('Predictions', ascending = False).
                       iloc[:num_recommendations, :-1]
                      )

    return user_full, recommendations

In [ ]:
already_rated, predictions = recommend_movies(preds_df, 342, anime_info, anime_data, 10)

In [ ]:
already_rated.head(10)

In [ ]:
predictions